In [ ]:
# 如何标记和引用时间序列数据取决于应用程序，并且您可能有以下其中一项：
# · 时间戳，具体的时刻。
# · 固定的时间区间，例如2007的1月或整个2010年。
# · 时间间隔，由开始和结束时间戳表示。时间区间可以被认为是间隔的特殊情况。
# · 实验时间或消耗时间。每个时间戳是相对于特定开始时间的时间的量度（例如，自从被放置在烤箱中每秒烘烤的饼干的直径）。

In [ ]:
# 11.1 日期和时间数据的类型及工具
# datetime、time和calendar模块是开始处理时间数据的主要内容。datetime.datetime类型，或简写为datetime，是广泛使用的：

In [1]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
from dateutil.parser import parse
import numpy as np

In [3]:
now=datetime.now()
now  # 返回当前时间

datetime.datetime(2023, 10, 24, 20, 52, 46, 654568)

In [4]:
now.year,now.month,now.day

(2023, 10, 24)

In [9]:
# delta返回当前的时间差
delta=datetime(2011,1,1)-datetime(2008,6,24,8,15)
delta

datetime.timedelta(days=920, seconds=56700)

In [10]:
delta.days  #926

920

In [11]:
delta.seconds  #56700

56700

In [12]:
# 你可以为一个datetime对象加上（或减去）一个timedelta或其整数倍来产生一个新的datetime对象：
start=datetime(2011,1,7)
start+timedelta(12)  #2011-01-19 00:00:00

datetime.datetime(2011, 1, 19, 0, 0)

In [13]:
start-2*timedelta(12) #2010-12-14 00:00:00

datetime.datetime(2010, 12, 14, 0, 0)

In [14]:
# 11.1.1 字符串与datetime互相转换
stamp=datetime(2011,1,3)
str(stamp) #2011-01-03 00:00:00

'2011-01-03 00:00:00'

In [15]:
stamp.strftime('%Y-%m-%d')  #2011-01-03

'2011-01-03'

In [16]:
# 你可以使用datetime.srtptime和这些格式代码，将字符串转换日期：
value='2011-01-03'
datetime.strptime(value,'%Y-%m-%d')                   #2011-01-03 00:00:00

datetime.datetime(2011, 1, 3, 0, 0)

In [17]:
datestrs=['7/6/2011','8/6/2011']
[datetime.strptime(x,'%m/%d/%Y') for x in datestrs]   #[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [18]:
# datetime.strptime是在已知格式的情况下转换日期的好方式。然而，每次都必须编写一个格式代码可能有点烦人，特别是对于通用日期格式。
# 在这种情况下，你可以使用第三方dateutil包的parser.parse方法（这个包在安装pandas时已经自动安装）
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [19]:
# dateutil能够解析大部分分类可以理解的日期表示
parse('Jan 31,1997 10:45 PM')

datetime.datetime(2023, 1, 31, 22, 45)

In [20]:
# 在国际场合下，日期出现在月份之前很常见，因此你可以传递dayfirst=True来表明这种情况：
parse('6/12/2011',dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [21]:
# pandas主要是面向处理日期数组的，无论是用作轴索引还是用作DataFrame中的列。to_datetime方法可以转换很多不同的日期表示格式
datestrs=['2011-07-06 12:00:00','2011-08-06 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

In [23]:
# to_datetime方法还可以处理那些被认为是缺失值的值(None,空字符串)
idx=pd.to_datetime(datestrs+[None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [24]:
idx[2]   # NaT  NaT（Not a time）是pandas中时间戳数据的是null值。

NaT

In [25]:
pd.isnull(idx)    #[False False  True]

array([False, False,  True])

In [27]:
# 11.2时间序列基础
# pandas中的基础时间序列种类是由时间戳索引的Series，在pandas外部则通常表示为Python字符串或datetime对象：
dates=[datetime(2011,1,2),datetime(2011,1,5),datetime(2011,1,7),
       datetime(2011,1,8),datetime(2011,1,10),datetime(2011,1,12)]
ts=pd.Series(np.random.randn(6),index=dates)
ts

2011-01-02    0.271580
2011-01-05    1.010289
2011-01-07    0.423683
2011-01-08   -0.744459
2011-01-10   -0.303909
2011-01-12   -0.510376
dtype: float64

In [28]:
# 这些datetime对象可以被放入DatetimeIndex中：
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [29]:
# 和其他Series类似，不同索引的时间序列之间的算术运算在日期上自动对齐：
ts+ts[::2]  #ts[::2]会将ts中每隔一个的元素选择出。

2011-01-02    0.543160
2011-01-05         NaN
2011-01-07    0.847365
2011-01-08         NaN
2011-01-10   -0.607818
2011-01-12         NaN
dtype: float64

In [31]:
# pandas使用NumPy的datetime64数据类型在纳秒级的分辨率下存储时间戳：
ts.index.dtype

dtype('<M8[ns]')

In [33]:
stamp=ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

In [35]:
# 11.2.1 索引、选择、子集
# 当你基于标签进行索引和选择时，时间序列的行为和其他的pandas.Series类似：
stamp=ts.index[2]
ts[stamp]

0.42368266541131383

In [36]:
# 还可以传递一个能解释为日期的字符串
ts['1/10/2011']

-0.3039092188887273

In [37]:
ts['20110110']

-0.3039092188887273

In [40]:
# 对一个长的时间序列，可以传递一个年份或一个年份和月份来轻松地选择数据的切片：
longer_ts=pd.Series(np.random.randn(1000),index=pd.date_range('1/1/2000',periods=1000))
longer_ts

2000-01-01   -0.318313
2000-01-02   -0.427608
2000-01-03    2.228414
2000-01-04    1.175354
2000-01-05    1.051563
                ...   
2002-09-22    0.802631
2002-09-23   -0.160985
2002-09-24   -1.390441
2002-09-25   -0.710825
2002-09-26   -1.153323
Freq: D, Length: 1000, dtype: float64

In [41]:
longer_ts['2001']

2001-01-01   -1.027539
2001-01-02   -0.641867
2001-01-03    0.178581
2001-01-04    0.614443
2001-01-05    1.514661
                ...   
2001-12-27    1.154761
2001-12-28   -0.097984
2001-12-29   -0.913858
2001-12-30    1.075727
2001-12-31   -1.054118
Freq: D, Length: 365, dtype: float64

In [42]:
# 字符串’2001’被解释为一个年份，并选择了相应的时间区间。如果你指定了月份也是有效的：
longer_ts['2001-05']

2001-05-01    0.785198
2001-05-02   -0.146842
2001-05-03   -1.360611
2001-05-04    1.204388
2001-05-05    0.623133
2001-05-06   -1.200034
2001-05-07    0.022667
2001-05-08   -0.151890
2001-05-09    2.297447
2001-05-10   -0.536667
2001-05-11   -0.506555
2001-05-12    1.022165
2001-05-13    0.219418
2001-05-14    0.818199
2001-05-15   -1.289150
2001-05-16    1.393778
2001-05-17   -0.199287
2001-05-18    0.440229
2001-05-19   -1.249942
2001-05-20    0.974777
2001-05-21    0.835739
2001-05-22   -0.182810
2001-05-23   -0.335357
2001-05-24    0.525352
2001-05-25    1.983179
2001-05-26   -0.573558
2001-05-27   -0.192986
2001-05-28    1.333590
2001-05-29   -0.230732
2001-05-30   -0.836485
2001-05-31   -0.875925
Freq: D, dtype: float64

In [44]:
# 也可以使用datetime对象进行切片
ts[datetime(2011,1,7):]

2011-01-07    0.423683
2011-01-08   -0.744459
2011-01-10   -0.303909
2011-01-12   -0.510376
dtype: float64

In [45]:
# 大部分时间序列数据是按时间顺序排序的，可以使用不包含在时间序列中的时间戳进行切片，以执行范围查询
ts

2011-01-02    0.271580
2011-01-05    1.010289
2011-01-07    0.423683
2011-01-08   -0.744459
2011-01-10   -0.303909
2011-01-12   -0.510376
dtype: float64

In [46]:
ts['1/6/2011':'1/11/2011']

2011-01-07    0.423683
2011-01-08   -0.744459
2011-01-10   -0.303909
dtype: float64

In [48]:
# 和之前一样，你可以传递一个字符串的日期、datetime对象或者时间戳。请记住通过这种方式的切片产生了原时间序列的视图，类似于NumPy的数组。这意味着没有数据被复制，并且在切片上的修改会反映在原始数据上
# 有一个等价实例方法，truncate，它可以在两个日期间对Series进行切片：
ts.truncate(after='1/9/2011')

2011-01-02    0.271580
2011-01-05    1.010289
2011-01-07    0.423683
2011-01-08   -0.744459
dtype: float64

In [52]:
# 上面这些操作也都适用于DataFrame，并在其行上进行索引：
dates=pd.date_range('1/1/2000',periods=100,freq='W-WED')
long_df=pd.DataFrame(np.random.randn(100,4),index=dates,columns=['Colorado','Texas','New York','Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,0.041805,-1.233562,-0.233392,-1.470018
2001-05-09,1.356494,-1.603719,1.600172,1.851263
2001-05-16,-1.391093,0.455580,-1.133291,1.197910
2001-05-23,-0.444169,-0.303375,0.135583,-0.750504
2001-05-30,1.114743,0.273048,0.658442,-0.038720


In [54]:
# 11.2.2含有重复索引的时间序列
# 在某些应用中，可能会有多个数据观察值落在特定的时间戳上
dates=pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dup_ts=pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [55]:
# 通过检查索引的is_unique属性，我们可以看出索引并不是唯一的：
dup_ts.index.is_unique

False

In [56]:
# 对上面的Series进行索引，结果是标量值还是Series切片取决于是否有时间戳是重复的：
dup_ts['1/3/2000']

4

In [57]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [58]:
# 假设你想要聚合含有非唯一时间戳的数据。一种方式就是使用groupby并传递level=0：
grouped=dup_ts.groupby(level=0)
grouped.mean()

2000-01-01    0.0
2000-01-02    2.0
2000-01-03    4.0
dtype: float64

In [59]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

In [60]:
# 11.3 日期范围、频率和移位
# pandas的通用时间序列是不规则的，即时间序列的频率不是固定的。对于很多应用来说，这足够了。然而，经常有需要处理固定频率的场景，例如每日的、每月的或每15分钟，这意味着我们甚至需要在必要的时候向时间序列中引入缺失值。幸运的是， pandas拥有一整套标准的时间序列频率和工具用于重新采样、推断频率以及生成固定频率的数据范围。例如，你可以通过调用resample方法将样本时间序列转换为固定的每日频率数据
ts

2011-01-02    0.271580
2011-01-05    1.010289
2011-01-07    0.423683
2011-01-08   -0.744459
2011-01-10   -0.303909
2011-01-12   -0.510376
dtype: float64

In [62]:
# 字符串‘D’解释为每日频率
resampler=ts.resample('D')
# 频率转换，称为重新采样，在11.6讲解

In [63]:
# 11.3.1生成日期范围
# pandas.date_range是用于根据特定频率生成指定长度的DatetimeIndex：
index=pd.date_range('2012-04-01','2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [64]:
# 默认情况下，date_range生成的是每日的时间戳。如果你只传递一个起始或结尾日期，你必须传递一个用于生成范围的数字：
pd.date_range(start='2012-04-01',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [65]:
pd.date_range(end='2012-06-01',periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [66]:
# 开始日期和结束日期严格定义了生成日期索引的边界。例如，如果你需要一个包含每月最后业务日期的时间索引，你可以传递’BM’频率（business end of month ，月度业务结尾；参考表11-4的频率列表），只有落在或在日期范围内的日期会被包括：
pd.date_range('2000-01-01','2000-12-01',freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [67]:
# 默认情况下，date_range保留开始或结束时间戳的时间（如果有的话）：
pd.date_range('2012-05-02 12:56:31',periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [70]:
# 有时候你会获得包含时间信息的开始日期或结束日期，但是你想要生成的是标准化为零点的时间戳。有一个normalize选项可以实现这个功能：
pd.date_range('2012-06-12 12:56:21',periods=5,normalize=True)

DatetimeIndex(['2012-06-12', '2012-06-13', '2012-06-14', '2012-06-15',
               '2012-06-16'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
11.3.2 频率和日期偏置